# サポートベクターマシン 実際のデータによるデモ
UC Irvine Machine Learning Repository で公開されている Adult データセットを用いてMADlibのサポートベクターマシンを試してみる

## 前準備

In [431]:
# ipython-sqlをロード
%reload_ext sql
# 文字コードをUTF-8にセット
# (データベースの文字コードもUTF-8にセットしておくこと)
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

## PostgreSQLに接続＆接続確認

In [435]:
%sql postgresql://postgres@centos72/postgres
%sql SELECT version();

version
"PostgreSQL 9.6.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-4), 64-bit"


## MADlibが正常にインストールされているか確認

In [436]:
%sql SELECT madlib.version();

version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-8-g82e56a4, cmake configuration time: 2016年 11月 22日 火曜日 00:25:23 UTC, build type: RelWithDebInfo, build system: Linux-3.10.0-327.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


## AdultデータをローカルPCにダウンロード

In [434]:
#import urllib
#urllib.urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', '/Users/Masanori/jupyter/MADlib/adult_dataset/adult.data')
#urllib.urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', '/Users/Masanori/jupyter/MADlib/adult_dataset/adult.test')

## テーブルを作成しテストデータをPostgreSQLにコピー

In [493]:
# pandasを使ってデータを読み込む 
import pandas as pd
from sqlalchemy import create_engine

adult_data = pd.read_csv(
                             'adult_dataset/adult.data'
                            ,header=None
                            ,names=[
                                 'age'
                                ,'workclass'
                                , 'fnlwgt'
                                , 'education'
                                , 'education_num'
                                , 'marital_status'
                                , 'occupation'
                                , 'relationship'
                                , 'race'
                                , 'sex'
                                , 'capital_gain'
                                , 'capital_loss'
                                , 'hours_per_week'
                                , 'native_country'
                                , 'class'
                            ]
                        )
# ipython-sql の機能を使ってPostgreSQLにデータを送信する
# テーブルは自動作成される
%sql DROP TABLE IF EXISTS adult_data;
%sql PERSIST adult_data
%sql SELECT * FROM adult_data LIMIT 5;

index,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## データが正常にPostgreSQLに送信されたことを確認
ローカルPCのデータの件数とPostgreSQLの対象テーブルのレコード件数を比較

In [441]:
adult_data['age'].count()

32561

In [446]:
%sql SELECT count(1) FROM adult_data;

count
32561


## Adultデータセットのテキストの属性情報を数値に変換する
サポートベクターマシンのインプットデータとしてテキストは使えない模様

In [455]:
%%sql

DROP TABLE IF EXISTS adult_data_parsed;
CREATE TABLE 
    adult_data_parsed
AS
    SELECT
         index
        ,age::FLOAT8
        ,CASE workclass
                    WHEN ' Private' THEN '1'::FLOAT8
                    WHEN ' Self-emp-not-inc' THEN '2'::FLOAT8
                    WHEN ' Self-emp-inc' THEN '3'::FLOAT8
                    WHEN ' Federal-gov' THEN '4'::FLOAT8
                    WHEN ' Local-gov' THEN '5'::FLOAT8
                    WHEN ' State-gov' THEN '6'::FLOAT8
                    WHEN ' Without-pay' THEN '7'::FLOAT8
                    WHEN ' Never-worked' THEN '8'::FLOAT8
                END AS workclass
        ,CASE education
                    WHEN ' Bachelors' THEN '1'::FLOAT8
                    WHEN ' Some-college' THEN '2'::FLOAT8
                    WHEN ' 11th' THEN '3'::FLOAT8
                    WHEN ' HS-grad' THEN '4'::FLOAT8
                    WHEN ' Prof-school' THEN '5'::FLOAT8
                    WHEN ' Assoc-acdm' THEN '6'::FLOAT8
                    WHEN ' Assoc-voc' THEN '7'::FLOAT8
                    WHEN ' 9th' THEN '8'::FLOAT8
                    WHEN ' 7th-8th' THEN '9'::FLOAT8
                    WHEN ' 12th' THEN '10'::FLOAT8
                    WHEN ' Masters' THEN '11'::FLOAT8
                    WHEN ' 1st-4th' THEN '12'::FLOAT8
                    WHEN ' 10th' THEN '13'::FLOAT8
                    WHEN ' Doctorate' THEN '14'::FLOAT8
                    WHEN ' 5th-6th' THEN '15'::FLOAT8
                    END AS education
        ,education_num::FLOAT8
        ,CASE marital_status
                    WHEN ' Married-civ-spouse' THEN '1'::FLOAT8
                    WHEN ' Divorced' THEN '2'::FLOAT8
                    WHEN ' Never-married' THEN '3'::FLOAT8
                    WHEN ' Separated' THEN '4'::FLOAT8
                    WHEN ' Widowed' THEN '5'::FLOAT8
                    WHEN ' Married-spouse-absent' THEN '6'::FLOAT8
                    WHEN ' Married-AF-spouse' THEN '7'::FLOAT8
                    END AS marital_status
        ,CASE occupation
                    WHEN ' Tech-support' THEN '1'::FLOAT8
                    WHEN ' Craft-repair' THEN '2'::FLOAT8
                    WHEN ' Other-service' THEN '3'::FLOAT8
                    WHEN ' Sales' THEN '4'::FLOAT8
                    WHEN ' Exec-managerial' THEN '5'::FLOAT8
                    WHEN ' Prof-specialty' THEN '6'::FLOAT8
                    WHEN ' Handlers-cleaners' THEN '7'::FLOAT8
                    WHEN ' Machine-op-inspct' THEN '8'::FLOAT8
                    WHEN ' Adm-clerical' THEN '9'::FLOAT8
                    WHEN ' Farming-fishing' THEN '10'::FLOAT8
                    WHEN ' Transport-moving' THEN '11'::FLOAT8
                    WHEN ' Priv-house-serv' THEN '12'::FLOAT8
                    WHEN ' Protective-serv' THEN '13'::FLOAT8
                    WHEN ' Armed-Forces' THEN '14'::FLOAT8
                    END AS occupation
        ,CASE relationship
                    WHEN ' Wife' THEN '1'::FLOAT8
                    WHEN ' Own-child' THEN '2'::FLOAT8
                    WHEN ' Husband' THEN '3'::FLOAT8
                    WHEN ' Not-in-family' THEN '4'::FLOAT8
                    WHEN ' Other-relative' THEN '5'::FLOAT8
                    WHEN ' Unmarried' THEN '6'::FLOAT8
                    END AS relationship
        ,CASE race
                    WHEN ' White' THEN '1'::FLOAT8
                    WHEN ' Asian-Pac-Islander' THEN '2'::FLOAT8
                    WHEN ' Amer-Indian-Eskimo' THEN '3'::FLOAT8
                    WHEN ' Other' THEN '4'::FLOAT8
                    WHEN ' Black' THEN '5'::FLOAT8
                    END AS race
        ,CASE sex 
                    WHEN ' Female' THEN '1'::FLOAT8
                    WHEN ' Male' THEN '2'::FLOAT8
                    END AS sex
        ,capital_gain::FLOAT8
        ,capital_loss::FLOAT8
        ,hours_per_week::FLOAT8
        ,CASE native_country
                    WHEN ' United-States' THEN '1'::FLOAT8
                    WHEN ' Cambodia' THEN '2'::FLOAT8
                    WHEN ' England' THEN '3'::FLOAT8
                    WHEN ' Puerto-Rico' THEN '4'::FLOAT8
                    WHEN ' Canada' THEN '5'::FLOAT8
                    WHEN ' Germany' THEN '6'::FLOAT8
                    WHEN ' Outlying-US(Guam-USVI-etc)' THEN '7'::FLOAT8
                    WHEN ' India' THEN '8'::FLOAT8
                    WHEN ' Japan' THEN '9'::FLOAT8
                    WHEN ' Greece' THEN '10'::FLOAT8
                    WHEN ' South, China' THEN '11'::FLOAT8
                    WHEN ' Cuba' THEN '12'::FLOAT8
                    WHEN ' Iran' THEN '13'::FLOAT8
                    WHEN ' Honduras' THEN '14'::FLOAT8
                    WHEN ' Philippines' THEN '15'::FLOAT8
                    WHEN ' Italy' THEN '16'::FLOAT8
                    WHEN ' Poland' THEN '17'::FLOAT8
                    WHEN ' Vietnam' THEN '18'::FLOAT8
                    WHEN ' Mexico' THEN '19'::FLOAT8
                    WHEN ' Portugal' THEN '20'::FLOAT8
                    WHEN ' Ireland' THEN '21'::FLOAT8
                    WHEN ' France' THEN '22'::FLOAT8
                    WHEN ' Dominican-Republic' THEN '23'::FLOAT8
                    WHEN ' Laos' THEN '24'::FLOAT8
                    WHEN ' Ecuador' THEN '25'::FLOAT8
                    WHEN ' Taiwan' THEN '26'::FLOAT8
                    WHEN ' Haiti' THEN '27'::FLOAT8
                    WHEN ' Columbia' THEN '28'::FLOAT8
                    WHEN ' Hungary' THEN '29'::FLOAT8
                    WHEN ' Guatemala' THEN '30'::FLOAT8
                    WHEN ' Nicaragua' THEN '31'::FLOAT8
                    WHEN ' Scotland' THEN '32'::FLOAT8
                    WHEN ' Thailand' THEN '33'::FLOAT8
                    WHEN ' Yugoslavia' THEN '34'::FLOAT8
                    WHEN ' El-Salvador' THEN '35'::FLOAT8
                    WHEN ' Trinadad&Tobago' THEN '36'::FLOAT8
                    WHEN ' Peru' THEN '37'::FLOAT8
                    WHEN ' Hong' THEN '38'::FLOAT8
                    WHEN ' Holand-Netherlands' THEN '39'::FLOAT8
                    END AS native_country
        ,CASE class
                    WHEN ' >50K' THEN '0'::FLOAT8
                    WHEN ' <=50K' THEN '1'::FLOAT8
                    END AS class
    FROM adult_data
;
SELECT * FROM adult_data_parsed LIMIT 5;

index,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39.0,6.0,1.0,13.0,3.0,9.0,4.0,1.0,2.0,2174.0,0.0,40.0,1.0,1.0
1,50.0,2.0,1.0,13.0,1.0,5.0,3.0,1.0,2.0,0.0,0.0,13.0,1.0,1.0
2,38.0,1.0,4.0,9.0,2.0,7.0,4.0,1.0,2.0,0.0,0.0,40.0,1.0,1.0
3,53.0,1.0,3.0,7.0,1.0,7.0,3.0,5.0,2.0,0.0,0.0,40.0,1.0,1.0
4,28.0,1.0,1.0,13.0,1.0,6.0,1.0,5.0,1.0,0.0,0.0,40.0,12.0,1.0


## テストデータもPostgreSQLにロード
作成したモデルを評価するためのデータセットも提供されている。PostgreSQLに送信する。

In [444]:
adult_test = pd.read_csv(
                            'adult_dataset/adult.test'
                            , header=None
                            , skiprows=1
                            ,names=[
                                'age'
                                , 'workclass'
                                , 'fnlwgt'
                                , 'education'
                                , 'education_num'
                                ,'marital_status'
                                , 'occupation'
                                , 'relationship'
                                , 'race'
                                , 'sex'
                                ,'capital_gain'
                                , 'capital_loss'
                                , 'hours_per_week'
                                , 'native_country'
                                , 'class'
                            ]
                        )
%sql DROP TABLE IF EXISTS adult_test;
%sql PERSIST adult_test

u'Persisted adult_test'

In [445]:
adult_test['age'].count()

16281

In [447]:
%sql SELECT count(1) FROM adult_test;

count
16281


In [459]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed;
CREATE TABLE 
    adult_test_parsed
AS
    SELECT
        index
        ,age::FLOAT8
        ,CASE workclass
                    WHEN ' Private' THEN '1'::FLOAT8
                    WHEN ' Self-emp-not-inc' THEN '2'::FLOAT8
                    WHEN ' Self-emp-inc' THEN '3'::FLOAT8
                    WHEN ' Federal-gov' THEN '4'::FLOAT8
                    WHEN ' Local-gov' THEN '5'::FLOAT8
                    WHEN ' State-gov' THEN '6'::FLOAT8
                    WHEN ' Without-pay' THEN '7'::FLOAT8
                    WHEN ' Never-worked' THEN '8'::FLOAT8
                    END AS workclass
        ,CASE education
                    WHEN ' Bachelors' THEN '1'::FLOAT8
                    WHEN ' Some-college' THEN '2'::FLOAT8
                    WHEN ' 11th' THEN '3'::FLOAT8
                    WHEN ' HS-grad' THEN '4'::FLOAT8
                    WHEN ' Prof-school' THEN '5'::FLOAT8
                    WHEN ' Assoc-acdm' THEN '6'::FLOAT8
                    WHEN ' Assoc-voc' THEN '7'::FLOAT8
                    WHEN ' 9th' THEN '8'::FLOAT8
                    WHEN ' 7th-8th' THEN '9'::FLOAT8
                    WHEN ' 12th' THEN '10'::FLOAT8
                    WHEN ' Masters' THEN '11'::FLOAT8
                    WHEN ' 1st-4th' THEN '12'::FLOAT8
                    WHEN ' 10th' THEN '13'::FLOAT8
                    WHEN ' Doctorate' THEN '14'::FLOAT8
                    WHEN ' 5th-6th' THEN '15'::FLOAT8
                    END AS education
        ,education_num::FLOAT8
        ,CASE marital_status
                    WHEN ' Married-civ-spouse' THEN '1'::FLOAT8
                    WHEN ' Divorced' THEN '2'::FLOAT8
                    WHEN ' Never-married' THEN '3'::FLOAT8
                    WHEN ' Separated' THEN '4'::FLOAT8
                    WHEN ' Widowed' THEN '5'::FLOAT8
                    WHEN ' Married-spouse-absent' THEN '6'::FLOAT8
                    WHEN ' Married-AF-spouse' THEN '7'::FLOAT8
                    END AS marital_status
        ,CASE occupation
                    WHEN ' Tech-support' THEN '1'::FLOAT8
                    WHEN ' Craft-repair' THEN '2'::FLOAT8
                    WHEN ' Other-service' THEN '3'::FLOAT8
                    WHEN ' Sales' THEN '4'::FLOAT8
                    WHEN ' Exec-managerial' THEN '5'::FLOAT8
                    WHEN ' Prof-specialty' THEN '6'::FLOAT8
                    WHEN ' Handlers-cleaners' THEN '7'::FLOAT8
                    WHEN ' Machine-op-inspct' THEN '8'::FLOAT8
                    WHEN ' Adm-clerical' THEN '9'::FLOAT8
                    WHEN ' Farming-fishing' THEN '10'::FLOAT8
                    WHEN ' Transport-moving' THEN '11'::FLOAT8
                    WHEN ' Priv-house-serv' THEN '12'::FLOAT8
                    WHEN ' Protective-serv' THEN '13'::FLOAT8
                    WHEN ' Armed-Forces' THEN '14'::FLOAT8
                    END AS occupation
        ,CASE relationship
                    WHEN ' Wife' THEN '1'::FLOAT8
                    WHEN ' Own-child' THEN '2'::FLOAT8
                    WHEN ' Husband' THEN '3'::FLOAT8
                    WHEN ' Not-in-family' THEN '4'::FLOAT8
                    WHEN ' Other-relative' THEN '5'::FLOAT8
                    WHEN ' Unmarried' THEN '6'::FLOAT8
                    END AS relationship
        ,CASE race
                    WHEN ' White' THEN '1'::FLOAT8
                    WHEN ' Asian-Pac-Islander' THEN '2'::FLOAT8
                    WHEN ' Amer-Indian-Eskimo' THEN '3'::FLOAT8
                    WHEN ' Other' THEN '4'::FLOAT8
                    WHEN ' Black' THEN '5'::FLOAT8
                    END AS race
        ,CASE sex
                    WHEN ' Female' THEN '1'::FLOAT8
                    WHEN ' Male' THEN '2'::bigint::FLOAT8
                    END AS sex
        ,capital_gain::FLOAT8
        ,capital_loss::FLOAT8
        ,hours_per_week::FLOAT8
        ,CASE native_country
                    WHEN ' United-States' THEN '1'::FLOAT8
                    WHEN ' Cambodia' THEN '2'::FLOAT8
                    WHEN ' England' THEN '3'::FLOAT8
                    WHEN ' Puerto-Rico' THEN '4'::FLOAT8
                    WHEN ' Canada' THEN '5'::FLOAT8
                    WHEN ' Germany' THEN '6'::FLOAT8
                    WHEN ' Outlying-US(Guam-USVI-etc)' THEN '7'::FLOAT8
                    WHEN ' India' THEN '8'::FLOAT8
                    WHEN ' Japan' THEN '9'::FLOAT8
                    WHEN ' Greece' THEN '10'::FLOAT8
                    WHEN ' South, China' THEN '11'::FLOAT8
                    WHEN ' Cuba' THEN '12'::FLOAT8
                    WHEN ' Iran' THEN '13'::FLOAT8
                    WHEN ' Honduras' THEN '14'::FLOAT8
                    WHEN ' Philippines' THEN '15'::FLOAT8
                    WHEN ' Italy' THEN '16'::FLOAT8
                    WHEN ' Poland' THEN '17'::FLOAT8
                    WHEN ' Vietnam' THEN '18'::FLOAT8
                    WHEN ' Mexico' THEN '19'::FLOAT8
                    WHEN ' Portugal' THEN '20'::FLOAT8
                    WHEN ' Ireland' THEN '21'::FLOAT8
                    WHEN ' France' THEN '22'::FLOAT8
                    WHEN ' Dominican-Republic' THEN '23'::FLOAT8
                    WHEN ' Laos' THEN '24'::FLOAT8
                    WHEN ' Ecuador' THEN '25'::FLOAT8
                    WHEN ' Taiwan' THEN '26'::FLOAT8
                    WHEN ' Haiti' THEN '27'::FLOAT8
                    WHEN ' Columbia' THEN '28'::FLOAT8
                    WHEN ' Hungary' THEN '29'::FLOAT8
                    WHEN ' Guatemala' THEN '30'::FLOAT8
                    WHEN ' Nicaragua' THEN '31'::FLOAT8
                    WHEN ' Scotland' THEN '32'::FLOAT8
                    WHEN ' Thailand' THEN '33'::FLOAT8
                    WHEN ' Yugoslavia' THEN '34'::FLOAT8
                    WHEN ' El-Salvador' THEN '35'::FLOAT8
                    WHEN ' Trinadad&Tobago' THEN '36'::FLOAT8
                    WHEN ' Peru' THEN '37'::FLOAT8
                    WHEN ' Hong' THEN '38'::FLOAT8
                    WHEN ' Holand-Netherlands' THEN '39'::FLOAT8
                    END AS native_country
        ,CASE class 
                    WHEN ' >50K.' THEN '0'::FLOAT8
                    WHEN ' <=50K.' THEN '1'::FLOAT8
                    END AS class
    FROM adult_test
;
SELECT * FROM adult_test_parsed LIMIT 5;

index,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,25.0,1.0,3.0,7.0,3.0,8.0,2.0,5.0,2.0,0.0,0.0,40.0,1.0,1.0
1,38.0,1.0,4.0,9.0,1.0,10.0,3.0,1.0,2.0,0.0,0.0,50.0,1.0,1.0
2,28.0,5.0,6.0,12.0,1.0,13.0,3.0,1.0,2.0,0.0,0.0,40.0,1.0,0.0
3,44.0,1.0,2.0,10.0,1.0,8.0,3.0,5.0,2.0,7688.0,0.0,40.0,1.0,0.0
4,18.0,None,2.0,10.0,3.0,None,2.0,1.0,1.0,0.0,0.0,30.0,1.0,1.0


## 線形モデル作製
まずは線形モデルを作成、トレーニングを行う。ARRAYに記載しているカラムを独立変数として選択。

In [449]:
%%sql

DROP TABLE IF EXISTS
     adult_data_parsed_svm
    ,adult_data_parsed_svm_summary
;
SELECT
    madlib.svm_classification(
         'adult_data_parsed'
        ,'adult_data_parsed_svm'
        ,'class = 0.0'
        ,'ARRAY[
            age
            ,workclass
            ,education
            ,education_num
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,capital_gain
            ,capital_loss
            ,hours_per_week
            ,native_country
         ]'
    )
;
SELECT * FROM adult_data_parsed_svm;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.149209009007529, -0.0363348195001957, 0.27651499541279, 1.67171366788677, -33.9858870913864, 0.0278045141390653, 1.39414013066558, 0.183006182555921, -1.37951835928823, 0.00488959391793084, 0.00343621919868571, 0.166854534901831, -0.0824172601010893]",76077.9980464,364549.829656,100,29900,2661,"[False, True]"


## 線形モデルテスト
作成したモデルをテストデータを使って評価する。テストデータを使って所得が>50Kか予測させる。

In [452]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred;
SELECT
    madlib.svm_predict(
         'adult_data_parsed_svm'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred'
    )
;
SELECT * FROM adult_test_parsed_pred LIMIT 10;

index,prediction,decision_function
0,False,-77.9737555024
1,False,-2.05605209361
2,True,0.289557726197
3,True,36.5569540315
5,False,-75.2887505237
7,True,24.0024455574
8,False,-67.2993069221
9,False,-13.392109812
10,True,31.6298505376
11,True,1.69748525202


## Area under the ROC curve による予測結果の検証を行うため評価用のテーブルを作成する。

In [454]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_roc_input;
CREATE TABLE
    adult_test_parsed_pred_roc_input
AS
    SELECT 
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0'::FLOAT8
            WHEN False THEN '1'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred AS pre
        LEFT OUTER JOIN adult_test_parsed AS ob
            ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,0.0
0.0,0.0
1.0,1.0
0.0,0.0
1.0,1.0
1.0,1.0
0.0,0.0
1.0,0.0


## area_under_roc 算出

In [460]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_roc_input'
        ,'adult_test_parsed_pred_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_roc_output;

area_under_roc
0.75739088673387834842500000000000000000000


## 非線形モデル作製
n_componentの値を変えながら複数の非線形モデルを作成して評価する。

## n_components=10

In [462]:
%%sql

DROP TABLE IF EXISTS
    adult_data_parsed_svm_gaussian_10
    ,adult_data_parsed_svm_gaussian_10_summary
    ,adult_data_parsed_svm_gaussian_10_random
    ;
SELECT
    madlib.svm_classification(
        'adult_data_parsed'
        ,'adult_data_parsed_svm_gaussian_10'
        ,'class = 0.0'
        ,'ARRAY[
            workclass
            ,education
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,native_country
        ]'
        ,'gaussian'
        ,'n_components=10'
        ,''
        ,'init_stepsize=2,
            max_iter=200'
    )
;
SELECT * FROM adult_data_parsed_svm_gaussian_10;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-2.93651172290274, 0.341683783738794, -0.440133878862007, 0.257267436381406, 0.35899688525636, 0.134067316852449, -0.048281495957981, 0.505773126346411, -0.179603594680169, 0.686461643517295]",19719.5132261,106.026378711,160,29900,2661,"[False, True]"


In [463]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_10;
SELECT
    madlib.svm_predict(
        'adult_data_parsed_svm_gaussian_10'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred_gaussian_10'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_10 LIMIT 10;

index,prediction,decision_function
0,False,-1.32993233113
1,False,-1.09707010755
2,True,0.321027412741
3,False,-2.14599864169
5,False,-0.597154717257
7,False,-0.715990222365
8,False,-1.48916354527
9,False,-1.24085773818
10,False,-0.923307390366
11,True,0.111854319862


In [466]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_10_roc_input;
CREATE TABLE
    adult_test_parsed_pred_gaussian_10_roc_input
AS
    SELECT
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0.0'::FLOAT8
            WHEN False THEN '1.0'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred_gaussian_10 AS pre
            LEFT OUTER JOIN adult_test_parsed AS ob
                ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_gaussian_10_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,0.0
0.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0
1.0,1.0
0.0,1.0
1.0,0.0


In [469]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_10_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_gaussian_10_roc_input'
        ,'adult_test_parsed_pred_gaussian_10_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_10_roc_output;

area_under_roc
0.53230553708029699758500000000000000000000


## n_components=20

In [470]:
%%sql

DROP TABLE IF EXISTS
    adult_data_parsed_svm_gaussian_20
    ,adult_data_parsed_svm_gaussian_20_summary
    ,adult_data_parsed_svm_gaussian_20_random
;
SELECT
    madlib.svm_classification(
        'adult_data_parsed'
        ,'adult_data_parsed_svm_gaussian_20'
        ,'class = 0.0'
        ,'ARRAY[
            workclass
            ,education
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,native_country
        ]'
        ,'gaussian'
        ,'n_components=20'
        ,''
        ,'init_stepsize=2,
            max_iter=200'
    )
;
SELECT * FROM adult_data_parsed_svm_gaussian_20;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-3.86814810268387, 0.449707764693438, -0.755980138466978, 0.181375858849701, 0.829237319869862, 0.00458928801845787, 0.171058151350693, -0.513100445299128, -0.430969534150961, 0.105338585609318, -0.237240804803337, 0.359755473290516, -0.335599590033492, 0.135979395068039, -0.0112713606630472, -0.274565699889732, -0.931919949164275, -0.860925475812818, 0.321416675647694, -1.0238848578281]",18849.1150496,89.7174444837,183,29900,2661,"[False, True]"


In [471]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_20;
SELECT
    madlib.svm_predict(
        'adult_data_parsed_svm_gaussian_20'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred_gaussian_20'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_20 LIMIT 10;

index,prediction,decision_function
0,False,-1.17190698142
1,False,-1.18132717988
2,False,-0.0445688331689
3,False,-1.77818365967
5,False,-0.971939899064
7,False,-0.874358793898
8,False,-0.882414054576
9,False,-1.65001996431
10,False,-1.00112560513
11,False,-0.131453982087


In [472]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_20_roc_input;
CREATE TABLE
    adult_test_parsed_pred_gaussian_20_roc_input
AS
    SELECT
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0.0'::FLOAT8
            WHEN False THEN '1.0'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred_gaussian_20 AS pre
            LEFT OUTER JOIN adult_test_parsed AS ob
                ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_gaussian_20_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,1.0
0.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0


In [473]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_20_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_gaussian_20_roc_input'
        ,'adult_test_parsed_pred_gaussian_20_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_20_roc_output;

area_under_roc
0.52747277189892029985000000000000000000000


## n_components=30

In [474]:
%%sql

DROP TABLE IF EXISTS
    adult_data_parsed_svm_gaussian_30
    ,adult_data_parsed_svm_gaussian_30_summary
    ,adult_data_parsed_svm_gaussian_30_random
;
SELECT
    madlib.svm_classification(
        'adult_data_parsed'
        ,'adult_data_parsed_svm_gaussian_30'
        ,'class = 0.0'
        ,'ARRAY[
            workclass
            ,education
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,native_country
        ]'
        ,'gaussian'
        ,'n_components=30'
        ,''
        ,'init_stepsize=2,
            max_iter=200'
    )
;
SELECT * FROM adult_data_parsed_svm_gaussian_30;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-4.89899524001805, 0.502068590111403, -0.448432712468678, -0.264644263446127, 0.953312188977701, 0.542099805581305, -0.0491652636454902, -1.46433389026839, -0.694065328395931, 1.07884194353142, -0.539551708488271, 0.00681979333013868, -0.356586988637732, 0.0622683338217195, 0.277744926344081, -0.432933514181231, -2.21751016511408, -0.267193976008079, -0.510322287251929, -1.51443422598838, 0.0687754778661905, 0.711737438466957, 0.0931018297577553, 0.535800897591793, -0.246127314688611, 2.62781277064238, 0.144360687030306, 0.628837085118467, -0.296165387275386, 0.119662047659125]",17647.5505522,273.251910947,194,29900,2661,"[False, True]"


In [475]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_30;
SELECT
    madlib.svm_predict(
        'adult_data_parsed_svm_gaussian_30'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred_gaussian_30'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_30 LIMIT 10;

index,prediction,decision_function
0,False,-1.14259429013
1,False,-1.27152168076
2,False,-0.207596574986
3,False,-1.05631585258
5,False,-0.946978327983
7,False,-0.896884632668
8,False,-1.09335208809
9,False,-1.52032030881
10,False,-0.999999955925
11,True,0.154337355323


In [476]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_30_roc_input;
CREATE TABLE
    adult_test_parsed_pred_gaussian_30_roc_input
AS
    SELECT
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0.0'::FLOAT8
            WHEN False THEN '1.0'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred_gaussian_30 AS pre
            LEFT OUTER JOIN adult_test_parsed AS ob
                ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_gaussian_30_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,1.0
0.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0
1.0,1.0
0.0,1.0
1.0,0.0


In [477]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_30_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_gaussian_30_roc_input'
        ,'adult_test_parsed_pred_gaussian_30_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_30_roc_output;

area_under_roc
0.61805934273185341981500000000000000000000


## n_components=40

In [478]:
%%sql

DROP TABLE IF EXISTS
    adult_data_parsed_svm_gaussian_40
    ,adult_data_parsed_svm_gaussian_40_summary
    ,adult_data_parsed_svm_gaussian_40_random
;
SELECT
    madlib.svm_classification(
        'adult_data_parsed'
        ,'adult_data_parsed_svm_gaussian_40'
        ,'class = 0.0'
        ,'ARRAY[
            workclass
            ,education
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,native_country
        ]'
        ,'gaussian'
        ,'n_components=40'
        ,''
        ,'init_stepsize=2,
            max_iter=200'
    )
;
SELECT * FROM adult_data_parsed_svm_gaussian_40;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-4.88484314019201, 0.536204691911348, -1.27689416201869, -0.397085816795131, 1.16781310467534, 0.603824276083918, -0.0903523770323435, -1.61516267518376, -0.512712743985378, 1.58585108956316, -0.589430217592102, -0.694512246389889, -0.751021698709412, 0.770702693019932, 0.290135390494713, -0.429994127137603, -2.73049804431851, -0.423242109692993, -0.762852709237821, -1.89291388009236, -0.195074127605914, 0.431432035565612, 0.194014387093558, -0.292411939725685, -0.318556318845158, 1.87150422933071, -0.0919452127657904, 0.884950245249454, -0.184842415790891, 0.00634895368966422, -1.75886473453548, -0.693352426131809, 1.22592506673662, 0.0349468994590692, 0.829953797887951, 0.905333260868029, -0.235126637148866, 2.5739680904738, 0.678486783527511, 0.704528701931597]",16803.2914882,218.945039033,193,29900,2661,"[False, True]"


In [479]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_40;
SELECT
    madlib.svm_predict(
        'adult_data_parsed_svm_gaussian_40'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred_gaussian_40'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_40 LIMIT 10;

index,prediction,decision_function
0,False,-1.47216629147
1,False,-1.29243941586
2,False,-0.501686418949
3,False,-1.00116038862
5,False,-1.41009743909
7,False,-0.881206391154
8,False,-1.56859401587
9,False,-1.40380233705
10,False,-0.999999944356
11,False,-0.0198800864604


In [480]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_40_roc_input;
CREATE TABLE
    adult_test_parsed_pred_gaussian_40_roc_input
AS
    SELECT
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0.0'::FLOAT8
            WHEN False THEN '1.0'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred_gaussian_40 AS pre
            LEFT OUTER JOIN adult_test_parsed AS ob
                ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_gaussian_40_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,1.0
0.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0
1.0,1.0
0.0,1.0
1.0,1.0


In [481]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_40_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_gaussian_40_roc_input'
        ,'adult_test_parsed_pred_gaussian_40_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_40_roc_output;

area_under_roc
0.63083868411053258884500000000000000000000


## n_components=1000

In [482]:
%%sql

DROP TABLE IF EXISTS
    adult_data_parsed_svm_gaussian_1000
    ,adult_data_parsed_svm_gaussian_1000_summary
    ,adult_data_parsed_svm_gaussian_1000_random
;
SELECT
    madlib.svm_classification(
        'adult_data_parsed'
        ,'adult_data_parsed_svm_gaussian_1000'
        ,'class = 0.0'
        ,'ARRAY[
            workclass
            ,education
            ,marital_status
            ,occupation
            ,relationship
            ,race
            ,sex
            ,native_country
        ]'
        ,'gaussian'
        ,'n_components=1000'
        ,''
        ,'init_stepsize=2,
            max_iter=200'
    )
;
SELECT * FROM adult_data_parsed_svm_gaussian_1000;

[([-2.01824758710825, 0.142415330341722, -0.406952768146646, 0.323547279368753, 4.02757766368734, -0.736682098940432, -3.90502080708794, 2.09180936648094, -3.27643943025332, -1.78192192853058, -0.452869151390114, -5.79066527242473, -0.733466331531628, -5.68343719987323, -1.18998340218889, -0.88826142820505, -0.251904145544684, 3.88242628207631, -3.11886536657036, -1.90390904010437, 1.80267483350202, -3.69442774749305, -1.7395081067012, 0.991108483711886, -2.24051174401476, 2.09374165192408, -0.855821446232527, 0.565624422007473, -0.47204639432617, 0.673272521128769, -4.18526150826022, 4.50868541923335, 3.28084614597216, -1.83819039852294, 0.209500724447511, 5.91861796059941, 3.55913466083447, 7.81849911968738, 1.74133781387628, 10.1638054272517, -3.22369122918122, 2.37482750253116, -1.03779836354897, -3.66447170193058, 0.0350408516223365, 0.454528613577024, -1.19385053146103, 1.54694978640823, -1.24393227933923, -0.189506571607644, -0.356518047196438, 2.39130821865772, 7.86337290910466, 2.24370088600951, 0.857772317015588, -2.56506851657179, 0.413988496163076, 4.14905378313412, 1.37017732590804, -1.20695730132281, 1.57899457317195, 2.14692974288438, -2.42833341417327, 0.0430553944138281, 1.86211912116226, -0.00165507460752983, -1.21230431073205, 0.491551848744662, -0.24228190027641, 2.54654366294532, 1.63312376257851, 1.75189211712619, 0.348313469337073, 1.08010307456071, 1.4475806032459, 1.17555580372987, -5.24690391659617, 1.91843036934891, 2.72642738408904, -1.67502204100129, 1.03125566874432, -1.98654557542641, -1.14724673575079, 2.95963555880111, -3.1699074879827, 1.56512534015439, 0.280919254004124, 1.8138864319314, -0.799843848181833, -2.70183459617245, -2.6846761941514, 1.23619907454178, 4.83405871474735, -1.41270953582195, 0.163779400439891, 1.85126580362659, 1.64491558086739, -0.844527637738523, 1.93959737692736, 2.34865135591704, 0.145814495126528, -0.181799271294751, -1.54842093618882, 1.75290409031993, 1.08300599596827, 0.57155411177913, 0.399713835898495, -0.958369480412752, -1.24684291230027, 2.50776577918768, 6.05498231494876, -2.11599093772155, 0.260814542957255, 0.759979162442359, -7.18898952998077, 0.987106401364634, 0.498371515691701, -1.62388863377201, -1.62797444257821, -1.89017266630003, 0.819978163941632, 0.214647367818759, 1.91191696646278, 2.21717895881137, -1.21422616142453, 2.49551837719482, 0.647186660849757, 0.402355914533151, -1.31640062446456, -0.420828686024598, 0.0312071422413146, -0.447012810439768, 2.00283150929903, -0.711210981982425, -2.68528453998336, 2.17627634421483, -2.71857339812224, 5.34703351663374, -9.8147617098087, -4.76425179914937, -2.64893634963102, 2.02137004212944, -2.05950517597814, -1.61403641180907, 0.314959378199402, -1.27001755623414, 2.30945628940531, -0.282106657889014, 0.566783026711567, 2.91911067554183, -2.40459624791333, 0.58716948941795, -0.942129981400272, 2.42987958746258, 0.619646152537155, -0.405719120915164, 1.48610165557033, -0.461827462320926, 1.56500890135932, -1.76909198135687, -4.80317363328185, 0.524941154717851, -0.303775487819387, -2.43040103221232, 0.759644931200224, 0.829941848448204, 0.104320093244976, -2.53037580977074, 7.06568231220562, -4.07548770576049, -2.06353582162627, -1.63814180571116, 10.6384521520404, 0.854329429950901, -3.49722500315062, -1.35682982781711, 1.71879363405341, -1.73199583142812, 0.33882492701572, -1.54329725714477, -1.41149069417492, 0.857307173599383, 0.812726529482659, -5.60336383002867, -0.867887255580424, 1.6823572866346, -0.0649239896874719, 1.59075896785058, 0.417983035352976, -5.42700165820152, -0.608482751281484, -1.42562254429006, -2.94367515053271, -0.902577754366346, 0.321585649143886, -0.11001988050023, -1.30383490825469, 0.439353901291115, 0.980601987452019, 1.43484944868761, -0.0370424167254366, 1.55428858674486, 2.77958911896975, 0.127175718220849, 0.0781875801298774, 1.36049288702175, 1.241997993516, -1.37396500558922, 1.83205993866774, -1.46719466945584, 0.81721300634866, -3.38946476136628, -2.850927378886

In [483]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_1000;
SELECT
    madlib.svm_predict(
        'adult_data_parsed_svm_gaussian_1000'
        ,'adult_test_parsed'
        ,'index'
        ,'adult_test_parsed_pred_gaussian_1000'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_1000 LIMIT 10;

index,prediction,decision_function
0,False,-2.75168489053
1,False,-1.01947139686
2,True,1.00000001042
3,False,-1.03874735588
5,False,-2.38269391935
7,True,1.00000001097
8,False,-1.29193105377
9,False,-1.49512161449
10,False,-1.01893068037
11,True,1.09882108507


In [484]:
%%sql

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_1000_roc_input;
CREATE TABLE
    adult_test_parsed_pred_gaussian_1000_roc_input
AS
    SELECT
        ob.class::FLOAT8 AS observed
        ,CASE pre.prediction
            WHEN True THEN '0.0'::FLOAT8
            WHEN False THEN '1.0'::FLOAT8
            END AS prediction
    FROM adult_test_parsed_pred_gaussian_1000 AS pre
            LEFT OUTER JOIN adult_test_parsed AS ob
                ON pre.index = ob.index
;
SELECT * FROM adult_test_parsed_pred_gaussian_1000_roc_input LIMIT 10;

observed,prediction
1.0,1.0
1.0,1.0
0.0,0.0
0.0,1.0
1.0,1.0
0.0,0.0
1.0,1.0
1.0,1.0
0.0,1.0
1.0,0.0


In [485]:
%%sql 

DROP TABLE IF EXISTS adult_test_parsed_pred_gaussian_1000_roc_output;
SELECT
    madlib.area_under_roc(
        'adult_test_parsed_pred_gaussian_1000_roc_input'
        ,'adult_test_parsed_pred_gaussian_1000_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM adult_test_parsed_pred_gaussian_1000_roc_output;

area_under_roc
0.71222897399791103725000000000000000000000


## 結果をまとめてみる

In [492]:
%%sql
SELECT 
    aur.area_under_roc AS aur
    ,aur10.area_under_roc AS aur10
    ,aur20.area_under_roc AS aur20
    ,aur30.area_under_roc AS aur30
    ,aur40.area_under_roc AS aur40
    ,aur1000.area_under_roc AS aur1000
FROM
    adult_test_parsed_pred_roc_output AS aur
    ,adult_test_parsed_pred_gaussian_10_roc_output AS aur10
    ,adult_test_parsed_pred_gaussian_20_roc_output AS aur20
    ,adult_test_parsed_pred_gaussian_30_roc_output AS aur30
    ,adult_test_parsed_pred_gaussian_40_roc_output AS aur40
    ,adult_test_parsed_pred_gaussian_1000_roc_output AS aur1000
;

aur,aur10,aur20,aur30,aur40,aur1000
0.75739088673387834842500000000000000000000,0.53230553708029699758500000000000000000000,0.52747277189892029985000000000000000000000,0.61805934273185341981500000000000000000000,0.63083868411053258884500000000000000000000,0.71222897399791103725000000000000000000000


更なるチューニングの余地あり…